Problem 1

In [4]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 5  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra-eng/fra.txt'

# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 93
Max sequence length for inputs: 14
Max sequence length for outputs: 59





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 8000 samples, validate on 2000 samples
Epoch 1/5
8000/8000 [==============================] - 24s 3ms/step - loss: 1.1856 - acc: 0.7344 - val_loss: 1.0136 - val_acc: 0.7174
Epoch 2/5
8000/8000 [==============================] - 29s 4ms/step - loss: 0.8530 - acc: 0.7697 - val_loss: 0.8225 - val_acc: 0.7731
Epoch 3/5
8000/8000 [==============================] - 26s 3ms/step - loss: 0.6774 - acc: 0.8110 - val_loss: 0.6936 - val_acc: 0.8036
Epoch 4/5
8000/8000 [==============================] - 23s 3ms/step - loss: 0.5811 - acc: 0.8314 - val_loss: 0.6467 - val_acc: 0.8106
Epoch 5/5
8000/8000 [==============================] - 28s 4ms/step - loss: 0.5295 - acc: 0.8451 - val_loss: 0.5737 - val_acc: 0.8328
-
Input sentence:

Problem 2

In [5]:
import os
from PIL import Image
import numpy  as np
import tensorflow as tf
import keras
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt

In [6]:
!git clone https://github.com/pdollar/coco.git
!cd coco/PythonAPI/
!make
!python setup.py build
!python setup.py install
!cd ../../
!git clone https://github.com/yunjey/pytorch-tutorial.git

Cloning into 'coco'...
'make' is not recognized as an internal or external command,
operable program or batch file.
python: can't open file 'setup.py': [Errno 2] No such file or directory
python: can't open file 'setup.py': [Errno 2] No such file or directory
Cloning into 'pytorch-tutorial'...


In [7]:
# !cd pytorch-tutorial/tutorials/03-advanced/image_captioning/
os.chdir('./pytorch-tutorial/tutorials/03-advanced/image_captioning/')
# カレントディレクトリの取得
print(os.getcwd())

c:\Users\NewTech\OneDrive\Documents\Python Scripts\pytorch-tutorial\tutorials\03-advanced\image_captioning


In [21]:
import sys
print(sys.path)
sys.path.append('/lib/python3.7/site-packages')
from PIL import Image

['c:\\Users\\NewTech\\OneDrive\\Documents\\Python Scripts', 'c:\\Users\\NewTech\\anaconda3\\envs\\myenv\\python37.zip', 'c:\\Users\\NewTech\\anaconda3\\envs\\myenv\\DLLs', 'c:\\Users\\NewTech\\anaconda3\\envs\\myenv\\lib', 'c:\\Users\\NewTech\\anaconda3\\envs\\myenv', '', 'c:\\Users\\NewTech\\anaconda3\\envs\\myenv\\lib\\site-packages', 'c:\\Users\\NewTech\\anaconda3\\envs\\myenv\\lib\\site-packages\\win32', 'c:\\Users\\NewTech\\anaconda3\\envs\\myenv\\lib\\site-packages\\win32\\lib', 'c:\\Users\\NewTech\\anaconda3\\envs\\myenv\\lib\\site-packages\\Pythonwin', 'c:\\Users\\NewTech\\anaconda3\\envs\\myenv\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\NewTech\\.ipython', '/lib/python3.7/site-packages', './pytorch-tutorial/tutorials/03-advanced/image_captioning/', '/lib/python3.7/site-packages', '/lib/python3.7/site-packages']


In [27]:
!pip install -r requirements.txt

In [30]:
# 5. Test the model
!python sample.py --image='png/example.png'

<start> a group of giraffes standing next to each other . <end>


In [32]:
# ホットドッグを食べている人の画像
!python sample.py --image='png/images.jpeg'

<start> a man holding a hot dog in a bun . <end>


[Problem 3] 

Investigate what to do if you want to run with Keras
I ran the PyTorch implementation, but for some reason I want to run it in Keras. Investigate what steps will be taken and explain in as much detail as possible.

Especially this time I want to be able to use pre-trained weights for PyTorch in Keras, so be sure to mention that.

When using learned weights in keras

Convert and save PyTorch learned weights to ONNX format (using torch.onnx.export)

Convert ONNX format to a format that can be used by keras (using onnx2keras.onnx_to_keras)

Convert channel-first to channel-last (PyTorch channel-first, onnx_to_kera change_ordering=True)

(Depending on the layer, it may not be possible to convert from PyTorch to keras format, so adjust the layer so that it can be converted in advance)

Scratch similar encoders, decoders, etc. in keras